In [1]:
import numpy as np
import xarray as xr
from xhistogram.xarray import histogram as hist
from thermodynamics import calc_sigmantr
from dask.diagnostics import ProgressBar

In [2]:
# from distributed import Client, LocalCluster, progress
# cluster = LocalCluster(
#     threads_per_worker=8,
#     n_workers=1,
#     dashboard_address=3624,
#     processes=False,
# )
# client = Client(cluster)
# client

In [2]:
rootdir = '/home/ocean_shared_data1/DRAKKAR/ORCA025.L75-GJM189-S/'
config = 'ORCA025.L75-GJM189'
filepath = rootdir+'*/'+config+'_y2015m0[34]d*_gridT.nc'
filepath_hgrid = rootdir+'GRID/'+config+'_mesh_hgr.nc'
filepath_zgrid = rootdir+'GRID/'+config+'_mesh_zgr.nc'
lonrange = [-100,30]
latrange = [45,80]

In [3]:
%%time
# Load grid
ds_grid = xr.merge([xr.open_dataset(filepath_hgrid),xr.open_dataset(filepath_zgrid)],compat='override').squeeze()
# Change name of depth variable and drop time_counter
ds_grid = ds_grid.rename({'z':'deptht'}).assign_coords({'deptht':ds_grid.gdept_0.values}).drop('time_counter')
ds_grid = ds_grid

# Load data files
ds_T = xr.open_mfdataset(filepath,combine='by_coords',parallel=True)
# Merge with grid
ds_T = ds_T.update(ds_grid);

CPU times: user 351 ms, sys: 179 ms, total: 530 ms
Wall time: 10.2 s


In [5]:
# Select upper 50m
ds_T = ds_T.sel(deptht=slice(0,50))
# Isolate North Atlantic
NAcondition = (ds_T.nav_lon>=lonrange[0]) & (ds_T.nav_lon<=lonrange[1]) & (ds_T.nav_lat>=latrange[0]) & (ds_T.nav_lat<=latrange[1])
ds_T = ds_T.where(NAcondition,drop=True)

In [6]:
# Calculate neutral density
ds_T['vosigmantr'] = xr.apply_ufunc(calc_sigmantr, ds_T.votemper, ds_T.vosaline,
                      dask='parallelized', output_dtypes=[ds_T.votemper.dtype])

In [7]:
densities = np.arange(27.7,28,0.01)

In [8]:
%%time
volume = ds_T.e1t*ds_T.e2t*ds_T.e3t
TV = hist(
    ds_T.vosigmantr,
    bins=[densities],
    dim=['x','y','deptht'],
    weights=(volume*ds_T.votemper))
V = hist(
    ds_T.vosigmantr,
    bins=[densities],
    dim=['x','y','deptht'],
    weights=volume)
T = TV/V

CPU times: user 4.17 s, sys: 0 ns, total: 4.17 s
Wall time: 4.17 s


In [10]:
with ProgressBar():
    T.to_dataset().to_netcdf('data/processed/orca025_NA-z50_votemper-by-vosigmantr.nc', mode='w')

[########################################] | 100% Completed |  1hr 25min  0.3s


NameError: name 'T' is not defined